In [515]:
import json
import pandas as pd
from datetime import datetime
from listas import charlson_dict, delirium_list, dementia_list, depresyndr_list, uriincont_list, fecincont_list, pressulc_list, immob_list, conf_list, osteopor_list, sarcopenia_list, sleepdisturb_list, chrpain_list, iatrog_list, constipation_list, CVdisease_list, heartdisease_list, ND_list, DM_list, hepatopat_list, neopl_list, AcuteRenalF_list, dizsyn_list, VIH_list, psicosis_list

from auxiliary_functions import (obtenir_data_presencia_codi, restar_dates, codis_ICD, nombre_ingressos,
                                 dies_ingressat_total, interval_10_edat,
                                 sumar_barthel, sumar_emina, obtenir_ultima_clau, obtenir_pes_o_mitjana,
                                 canadenca_comparada, disfagia_mecvvs, extreure_valors_claus,
                                 extreure_valors_claus_simple, obtenir_valors_clau_interes, index_charlson,
                                 obtenir_pes_mes_antic, obtenir_pes_mes_nou, obtenir_data_pes_mes_antic,
                                 obtenir_primera_data_mecvv, obtenir_pes_coincident_mecvv, restar_columnes_object)

taula = pd.read_pickle('./data/processed/dataframe.pkl')
import scipy.stats as stats
from scipy.stats import ttest_ind
from scipy.stats import shapiro
from scipy.stats import mannwhitneyu
from scipy.stats import chi2_contingency


In [516]:
taula.head()

,edat,sexe,visitaDietista,labs,mna,emina,canadenca,pes,atcs,mecvvs,...,FGE CDK-EPI,Pes més antic,Pes més nou,Data pes més antic,Data primer MECVV,Pes coincident primer MECVV,Pèrdua pes entre ingressos,Pèrdua pes total,Data més antiga pneumònia,Dies entre primer ICD pneumònia i primer MECVV positiu
0,89,F,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'M01AE', 'dataInici': '2020-06-21 00...",[],...,None,None,None,None,NaT,None,None,None,2020-06-18,NaN
1,70,M,0,"[{'date': '20220524', 'name': 'PCR SARS-CoV2 S...",[],"[{'activitat': '0', 'estatMental': '0', 'humit...",[],"[{'valor': '64.7', 'data': '2022-10-04'}, {'va...","[{'codi': 'C09AA', 'dataInici': '2021-05-26 00...","[{'data': '20220930', 'pacientNoColabora': '',...",...,>90,70,53.7,2022-07-06,NaT,None,None,16.3,2022-09-29,NaN
2,96,M,0,[],[],"[{'activitat': '3', 'estatMental': '0', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-09 00...",[],...,None,None,None,None,NaT,None,None,None,2020-07-08,NaN
3,85,M,0,[],[],"[{'activitat': '2', 'estatMental': '1', 'humit...",[],[],"[{'codi': 'G04CA', 'dataInici': '2020-07-15 00...","[{'data': '20201204131810', 'disfagia': 'S', '...",...,None,None,None,None,2020-12-04,None,None,None,2020-12-02,2.0
4,93,M,0,"[{'date': '20220208', 'name': 'UREA Sèrum', 'l...","[{'imc': '0', 'anorexia': '1', 'perduaPes': '1...","[{'activitat': '2', 'estatMental': '1', 'humit...",[],"[{'valor': '49.900', 'data': '2022-06-02'}, {'...","[{'codi': 'N02AA', 'dataInici': '2022-02-01 00...","[{'data': '20220330172303', 'disfagiaConeguda'...",...,Proves afegides a un altre petició per extracc...,49.900,49.900,2022-06-02,NaT,None,None,0.0,2022-08-07,NaN


In [517]:
AMB_PA = taula[taula["PA diagnosticada"] == 1.0]
AMB_PA_MECVV = taula[
    (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] < 30)]
SENSE_PA = taula[
           (taula["Dies entre primer ICD pneumònia i primer MECVV positiu"] > 30) &
           (taula['P diagnosticada'] == 1.0)]
TOTAL = taula

# Síndromes geriàtrics

## Index de Charlson

In [518]:
Mitjana_charlson = AMB_PA['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 10.24 ± 10.73


In [519]:
Mitjana_charlson = AMB_PA_MECVV['Charlson'].mean()
desviacio_estandard_charlson = AMB_PA_MECVV['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 16.00 ± 11.60


In [520]:
Mitjana_charlson = SENSE_PA['Charlson'].mean()
desviacio_estandard_charlson = SENSE_PA['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 16.50 ± 12.83


In [521]:
Mitjana_charlson = TOTAL['Charlson'].mean()
desviacio_estandard_charlson = TOTAL['Charlson'].std()

resultat = f"{Mitjana_charlson:.2f} ± {desviacio_estandard_charlson:.2f}"
print("Mitjana ± Desviació Estàndard de l'índex de Charlson:", resultat)

Mitjana ± Desviació Estàndard de l'índex de Charlson: 11.62 ± 13.16


#### Pvalor

##### Normalitat

In [522]:
stat, p_value = shapiro(AMB_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7994
Valor p = 0.0020
El nombre de files no segueix una distribució normal.


In [523]:
stat, p_value = shapiro(AMB_PA_MECVV['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9651
Valor p = 0.8571
El nombre de files segueix una distribució normal


In [524]:
stat, p_value = shapiro(SENSE_PA['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9177
Valor p = 0.4113
El nombre de files segueix una distribució normal


In [525]:
stat, p_value = shapiro(TOTAL['Charlson'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.7836
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [526]:
stat, p_value_12 = mannwhitneyu(AMB_PA['Charlson'], AMB_PA_MECVV['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i AMB_PA_MECVV", p_value_12)

P-valor entre AMB_PA i AMB_PA_MECVV 0.23085687112868614


In [527]:
stat, p_value_13 = mannwhitneyu(AMB_PA['Charlson'], SENSE_PA['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i SENSE_PA", p_value_13)

P-valor entre AMB_PA i SENSE_PA 0.1977214805403812


In [528]:
stat, p_value_23 = ttest_ind(AMB_PA_MECVV['Charlson'], SENSE_PA['Charlson'], equal_var=False)
print("P-valor entre AMB_PA_MECVV i SENSE_PA", p_value_23)

P-valor entre AMB_PA_MECVV i SENSE_PA 0.9360013644802581


In [529]:
stat, p_value_14 = mannwhitneyu(AMB_PA['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA i TOTAL", p_value_14)

P-valor entre AMB_PA i TOTAL 0.8623665948573777


In [530]:
stat, p_value_24 = mannwhitneyu(AMB_PA_MECVV['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre AMB_PA_MECVV i TOTAL", p_value_24)

P-valor entre AMB_PA_MECVV i TOTAL 0.2016616772763471


In [531]:
stat, p_value_34 = mannwhitneyu(SENSE_PA['Charlson'], TOTAL['Charlson'], alternative='two-sided')
print("P-valor entre SENSE_PA i TOTAL", p_value_34)

P-valor entre SENSE_PA i TOTAL 0.19370672249585386


## Caigudes prèvies

In [532]:
caprev_AMB_PA= (AMB_PA['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA:.2f}%")

Percentatge de caigudes prèvies: 5.88%


In [533]:
caprev_AMB_PA_MECVV= (AMB_PA_MECVV['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_AMB_PA_MECVV:.2f}%")

Percentatge de caigudes prèvies: 0.00%


In [534]:
caprev_SENSE_PA= (SENSE_PA['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_SENSE_PA:.2f}%")

Percentatge de caigudes prèvies: 12.50%


In [535]:
caprev_TOTAL= (TOTAL['Caigudes prèvies'] == 1.0).mean() * 100
print(f"Percentatge de caigudes prèvies: {caprev_TOTAL:.2f}%")

Percentatge de caigudes prèvies: 6.00%


#### Pvalor

##### Normalitat

In [536]:
stat, p_value = shapiro(AMB_PA['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [537]:
stat, p_value = shapiro(AMB_PA_MECVV['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [538]:
stat, p_value = shapiro(SENSE_PA['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [539]:
stat, p_value = shapiro(TOTAL['Caigudes prèvies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2548
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [540]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=AMB_PA_MECVV['Caigudes prèvies'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i AMB_PA_MECVV.


In [541]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=SENSE_PA['Caigudes prèvies'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i SENSE_PA.


In [542]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Caigudes prèvies'], columns=SENSE_PA['Caigudes prèvies'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [543]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA i TOTAL.


In [544]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de caigudes prèvies en AMB_PA_MECVV i TOTAL.


In [545]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Caigudes prèvies'], columns=TOTAL['Caigudes prèvies'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de caigudes prèvies en SENSE_PA i TOTAL.


## Deliris

In [546]:
del_AMB_PA= (AMB_PA['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA:.2f}%")

Percentatge de deliris: 23.53%


In [547]:
del_AMB_PA_MECVV= (AMB_PA_MECVV['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_AMB_PA_MECVV:.2f}%")

Percentatge de deliris: 25.00%


In [548]:
del_SENSE_PA= (SENSE_PA['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_SENSE_PA:.2f}%")

Percentatge de deliris: 62.50%


In [549]:
del_TOTAL= (TOTAL['Deliris'] == 1.0).mean() * 100
print(f"Percentatge de deliris: {del_TOTAL:.2f}%")

Percentatge de deliris: 20.00%


#### Pvalor

##### Normalitat

In [550]:
stat, p_value = shapiro(AMB_PA['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [551]:
stat, p_value = shapiro(AMB_PA_MECVV['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [552]:
stat, p_value = shapiro(SENSE_PA['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [553]:
stat, p_value = shapiro(TOTAL['Deliris'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4904
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [554]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Deliris'], columns=AMB_PA_MECVV['Deliris'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i AMB_PA_MECVV.


In [555]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Deliris'], columns=SENSE_PA['Deliris'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i SENSE_PA.


In [556]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Deliris'], columns=SENSE_PA['Deliris'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [557]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Deliris'], columns=TOTAL['Deliris'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de deliris en AMB_PA i TOTAL.


In [558]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Deliris'], columns=TOTAL['Deliris'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de deliris en AMB_PA_MECVV i TOTAL.


In [559]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Deliris'], columns=TOTAL['Deliris'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de deliris en SENSE_PA i TOTAL.


## Demència

In [560]:
dem_AMB_PA= (AMB_PA['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_AMB_PA:.2f}%")

Percentatge de demència: 47.06%


In [561]:
dem_AMB_PA_MECVV= (AMB_PA_MECVV['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_AMB_PA_MECVV:.2f}%")

Percentatge de demència: 25.00%


In [562]:
dem_SENSE_PA= (SENSE_PA['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_SENSE_PA:.2f}%")

Percentatge de demència: 62.50%


In [563]:
dem_TOTAL= (TOTAL['Demència'] == 1.0).mean() * 100
print(f"Percentatge de demència: {dem_TOTAL:.2f}%")

Percentatge de demència: 32.00%


#### Pvalor

##### Normalitat

In [564]:
stat, p_value = shapiro(AMB_PA['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6418
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [565]:
stat, p_value = shapiro(AMB_PA_MECVV['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [566]:
stat, p_value = shapiro(SENSE_PA['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [567]:
stat, p_value = shapiro(TOTAL['Demència'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5879
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [568]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Demència'], columns=AMB_PA_MECVV['Demència'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de demència en AMB_PA i AMB_PA_MECVV.


In [569]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Demència'], columns=SENSE_PA['Demència'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de demència en AMB_PA i SENSE_PA.


In [570]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Demència'], columns=SENSE_PA['Demència'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [571]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Demència'], columns=TOTAL['Demència'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 13.223042052469136
P-valor entre AMB_PA i TOTAL: 0.00027652843917458906
Hi ha diferències significatives entre les distribucions de demència en AMB_PA i TOTAL.


In [572]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Demència'], columns=TOTAL['Demència'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de demència en AMB_PA_MECVV i TOTAL.


In [573]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Demència'], columns=TOTAL['Demència'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de demència en SENSE_PA i TOTAL.


## Síndrome depressiu

In [574]:
depre_AMB_PA= (AMB_PA['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA:.2f}%")

Percentatge de síndrome depressiu: 23.53%


In [575]:
depre_AMB_PA_MECVV= (AMB_PA_MECVV['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_AMB_PA_MECVV:.2f}%")

Percentatge de síndrome depressiu: 12.50%


In [576]:
depre_SENSE_PA= (SENSE_PA['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_SENSE_PA:.2f}%")

Percentatge de síndrome depressiu: 12.50%


In [577]:
depre_TOTAL= (TOTAL['Síndrome depressiu'] == 1.0).mean() * 100
print(f"Percentatge de síndrome depressiu: {depre_TOTAL:.2f}%")

Percentatge de síndrome depressiu: 14.00%


#### Pvalor

##### Normalitat

In [578]:
stat, p_value = shapiro(AMB_PA['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [579]:
stat, p_value = shapiro(AMB_PA_MECVV['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [580]:
stat, p_value = shapiro(SENSE_PA['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [581]:
stat, p_value = shapiro(TOTAL['Síndrome depressiu'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4124
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [582]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=AMB_PA_MECVV['Síndrome depressiu'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.9600000000000002
P-valor entre AMB_PA i AMB_PA_MECVV: 0.32718687779030275
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i AMB_PA_MECVV.


In [583]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=SENSE_PA['Síndrome depressiu'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i SENSE_PA.


In [584]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Síndrome depressiu'], columns=SENSE_PA['Síndrome depressiu'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [585]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA i TOTAL.


In [586]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de síndrome depressiu en AMB_PA_MECVV i TOTAL.


In [587]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Síndrome depressiu'], columns=TOTAL['Síndrome depressiu'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de síndrome depressiu en SENSE_PA i TOTAL.


## Incontinència urinària

In [588]:
ic_AMB_PA= (AMB_PA['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_AMB_PA:.2f}%")

Percentatge d'incontinència urinària: 29.41%


In [589]:
ic_AMB_PA_MECVV= (AMB_PA_MECVV['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_AMB_PA_MECVV:.2f}%")

Percentatge d'incontinència urinària: 50.00%


In [590]:
ic_SENSE_PA= (SENSE_PA['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_SENSE_PA:.2f}%")

Percentatge d'incontinència urinària: 0.00%


In [591]:
ic_TOTAL= (TOTAL['Incont.uri'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència urinària: {ic_TOTAL:.2f}%")

Percentatge d'incontinència urinària: 16.00%


#### Pvalor

##### Normalitat

In [592]:
stat, p_value = shapiro(AMB_PA['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [593]:
stat, p_value = shapiro(AMB_PA_MECVV['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [594]:
stat, p_value = shapiro(SENSE_PA['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [595]:
stat, p_value = shapiro(TOTAL['Incont.uri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4412
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [596]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=AMB_PA_MECVV['Incont.uri'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.6666666666666665
P-valor entre AMB_PA i AMB_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.


In [597]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=SENSE_PA['Incont.uri'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i AMB_PA_MECVV.


In [598]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Incont.uri'], columns=SENSE_PA['Incont.uri'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [599]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA i TOTAL.


In [600]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions d'incontinència urinària en AMB_PA_MECVV i TOTAL.


In [601]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Incont.uri'], columns=TOTAL['Incont.uri'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència urinària en SENSE_PA i TOTAL.


## Incontinència fecal 

In [602]:
ifec_AMB_PA= (AMB_PA['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_AMB_PA:.2f}%")

Percentatge d'incontinència fecal: 29.41%


In [603]:
ifec_AMB_PA_MECVV= (AMB_PA_MECVV['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_AMB_PA_MECVV:.2f}%")

Percentatge d'incontinència fecal: 25.00%


In [604]:
ifec_SENSE_PA= (SENSE_PA['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_SENSE_PA:.2f}%")

Percentatge d'incontinència fecal: 0.00%


In [605]:
ifec_TOTAL= (TOTAL['Incont.fec'] == 1.0).mean() * 100
print(f"Percentatge d'incontinència fecal: {ifec_TOTAL:.2f}%")

Percentatge d'incontinència fecal: 12.00%


#### Pvalor

##### Normalitat

In [606]:
stat, p_value = shapiro(AMB_PA['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5785
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [607]:
stat, p_value = shapiro(AMB_PA_MECVV['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [608]:
stat, p_value = shapiro(SENSE_PA['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [609]:
stat, p_value = shapiro(TOTAL['Incont.fec'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3802
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [610]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=AMB_PA_MECVV['Incont.fec'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i AMB_PA_MECVV.


In [611]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=SENSE_PA['Incont.fec'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i SENSE_PA.


In [612]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Incont.fec'], columns=SENSE_PA['Incont.fec'])

chi2_23, p_value_23, dof_23, expected_13 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [613]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.")

Estadístic xi-quadrat entre AMB_PA i TOTAL: 12.524513888888887
P-valor entre AMB_PA i TOTAL: 0.00040164738150225485
Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA i TOTAL.


In [614]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.")

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions d'incontinència fecal en AMB_PA_MECVV i TOTAL.


In [615]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Incont.fec'], columns=TOTAL['Incont.fec'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.")

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'incontinència fecal en SENSE_PA i TOTAL.


## Úlceres de pressió

In [616]:
up_AMB_PA= (AMB_PA['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA:.2f}%")

Percentatge de úlceres de pressió: 5.88%


In [617]:
up_AMB_PA_MECVV= (AMB_PA_MECVV['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_AMB_PA_MECVV:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [618]:
up_SENSE_PA= (SENSE_PA['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_SENSE_PA:.2f}%")

Percentatge de úlceres de pressió: 0.00%


In [619]:
up_TOTAL= (TOTAL['Úlceres pressió'] == 1.0).mean() * 100
print(f"Percentatge de úlceres de pressió: {up_TOTAL:.2f}%")

Percentatge de úlceres de pressió: 2.00%


#### Pvalor

##### Normalitat

In [620]:
stat, p_value = shapiro(AMB_PA['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [621]:
stat, p_value = shapiro(AMB_PA_MECVV['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [622]:
stat, p_value = shapiro(SENSE_PA['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [623]:
stat, p_value = shapiro(TOTAL['Úlceres pressió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1254
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [624]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=AMB_PA_MECVV['Úlceres pressió'])

chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)

print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)

if p_value_12 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.")


Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i AMB_PA_MECVV.


In [625]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=SENSE_PA['Úlceres pressió'])

chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)

print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)

if p_value_13 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.")


Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i SENSE_PA.


In [626]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Úlceres pressió'], columns=SENSE_PA['Úlceres pressió'])

chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)

print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)

if p_value_23 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i SENSE_PA.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i SENSE_PA.")

ValueError: No data; `observed` has size 0.

In [627]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)

print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)
print("P-valor entre AMB_PA i TOTAL:", p_value_14)

if p_value_14 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.")


Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA i TOTAL.


In [628]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)

print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)

if p_value_24 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.")


Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en AMB_PA_MECVV i TOTAL.


In [629]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Úlceres pressió'], columns=TOTAL['Úlceres pressió'])

chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)

print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)

if p_value_34 < 0.05:
    print("Hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.")
else:
    print("No hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.")


Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'úlceres de pressió en SENSE_PA i TOTAL.


## Immobilitat

In [630]:
imm_AMB_PA= (AMB_PA['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA:.2f}%")

Percentatge d'immobilitat : 0.00%


In [631]:
imm_AMB_PA_MECVV= (AMB_PA_MECVV['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_AMB_PA_MECVV:.2f}%")

Percentatge d'immobilitat : 0.00%


In [632]:
imm_SENSE_PA= (SENSE_PA['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_SENSE_PA:.2f}%")

Percentatge d'immobilitat : 0.00%


In [633]:
imm_TOTAL= (TOTAL['Immobilitat'] == 1.0).mean() * 100
print(f"Percentatge d'immobilitat : {imm_TOTAL:.2f}%")

Percentatge d'immobilitat : 0.00%


#### Pvalor

##### Normalitat

In [634]:
stat, p_value = shapiro(AMB_PA['Immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [635]:
stat, p_value = shapiro(AMB_PA_MECVV['Immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [636]:
stat, p_value = shapiro(SENSE_PA['Immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [637]:
stat, p_value = shapiro(TOTAL['Immobilitat'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [638]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=AMB_PA_MECVV['Immobilitat']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i AMB_PA_MECVV.


In [639]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=SENSE_PA['Immobilitat']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i SENSE_PA.


In [640]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Immobilitat'], columns=SENSE_PA['Immobilitat']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i SENSE_PA.") 


ValueError: No data; `observed` has size 0.

In [641]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA i TOTAL.


In [642]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en AMB_PA_MECVV i TOTAL.


In [643]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Immobilitat'], columns=TOTAL['Immobilitat']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'immobilitat en SENSE_PA i TOTAL.


## Confusió

In [644]:
conf_AMB_PA= (AMB_PA['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA:.2f}%")

Percentatge de confusió : 23.53%


In [645]:
conf_AMB_PA_MECVV= (AMB_PA_MECVV['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_AMB_PA_MECVV:.2f}%")

Percentatge de confusió : 25.00%


In [646]:
conf_SENSE_PA= (SENSE_PA['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_SENSE_PA:.2f}%")

Percentatge de confusió : 62.50%


In [647]:
conf_TOTAL= (TOTAL['Confusió'] == 1.0).mean() * 100
print(f"Percentatge de confusió : {conf_TOTAL:.2f}%")

Percentatge de confusió : 20.00%


#### Pvalor

##### Normalitat

In [648]:
stat, p_value = shapiro(AMB_PA['Confusió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [649]:
stat, p_value = shapiro(AMB_PA_MECVV['Confusió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [650]:
stat, p_value = shapiro(SENSE_PA['Confusió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [651]:
stat, p_value = shapiro(TOTAL['Confusió'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4904
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [652]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Confusió'], columns=AMB_PA_MECVV['Confusió']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.34375
P-valor entre AMB_PA i AMB_PA_MECVV: 0.12578642463893994
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i AMB_PA_MECVV.


In [653]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Confusió'], columns=SENSE_PA['Confusió']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i SENSE_PA.


In [654]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Confusió'], columns=SENSE_PA['Confusió']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [655]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de confusió en AMB_PA i TOTAL.


In [656]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de confusió en AMB_PA_MECVV i TOTAL.


In [657]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Confusió'], columns=TOTAL['Confusió']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de confusió en SENSE_PA i TOTAL.


## Osteoporosi

In [658]:
ost_AMB_PA= (AMB_PA['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [659]:
ost_AMB_PA_MECVV= (AMB_PA_MECVV['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_AMB_PA_MECVV:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [660]:
ost_SENSE_PA= (SENSE_PA['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_SENSE_PA:.2f}%")

Percentatge d'osteoporosi: 0.00%


In [661]:
ost_TOTAL= (TOTAL['Osteoporosi'] == 1.0).mean() * 100
print(f"Percentatge d'osteoporosi: {ost_TOTAL:.2f}%")

Percentatge d'osteoporosi: 0.00%


#### Pvalor

##### Normalitat

In [662]:
stat, p_value = shapiro(AMB_PA['Osteoporosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [663]:
stat, p_value = shapiro(AMB_PA_MECVV['Osteoporosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [664]:
stat, p_value = shapiro(SENSE_PA['Osteoporosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [665]:
stat, p_value = shapiro(TOTAL['Osteoporosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [666]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=AMB_PA_MECVV['Osteoporosi']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i AMB_PA_MECVV.


In [667]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=SENSE_PA['Osteoporosi']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i SENSE_PA.


In [668]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Osteoporosi'], columns=SENSE_PA['Osteoporosi']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi  en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [669]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA i TOTAL.


In [670]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en AMB_PA_MECVV i TOTAL.


In [671]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Osteoporosi'], columns=TOTAL['Osteoporosi']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'osteoporosi en SENSE_PA i TOTAL.


## Sarcopènia

In [672]:
sarc_AMB_PA= (AMB_PA['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA:.2f}%")

Percentatge de sarcopènia: 0.00%


In [673]:
sarc_AMB_PA_MECVV= (AMB_PA_MECVV['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_AMB_PA_MECVV:.2f}%")

Percentatge de sarcopènia: 0.00%


In [674]:
sarc_SENSE_PA= (SENSE_PA['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_SENSE_PA:.2f}%")

Percentatge de sarcopènia: 12.50%


In [675]:
sarc_TOTAL= (TOTAL['Sarcopènia'] == 1.0).mean() * 100
print(f"Percentatge de sarcopènia: {sarc_TOTAL:.2f}%")

Percentatge de sarcopènia: 4.00%


#### Pvalor

##### Normalitat

In [676]:
stat, p_value = shapiro(AMB_PA['Sarcopènia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [677]:
stat, p_value = shapiro(AMB_PA_MECVV['Sarcopènia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [678]:
stat, p_value = shapiro(SENSE_PA['Sarcopènia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [679]:
stat, p_value = shapiro(TOTAL['Sarcopènia'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1982
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [680]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=AMB_PA_MECVV['Sarcopènia']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i AMB_PA_MECVV.


In [681]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=SENSE_PA['Sarcopènia']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia AMB_PA i SENSE_PA.


In [682]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Sarcopènia'], columns=SENSE_PA['Sarcopènia']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [683]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA i TOTAL.


In [684]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de sarcopènia en AMB_PA_MECVV i TOTAL.


In [685]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Sarcopènia'], columns=TOTAL['Sarcopènia']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de sarcopènia en SENSE_PA i TOTAL.


## Trastorns del son

In [686]:
ts_AMB_PA= (AMB_PA['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [687]:
ts_AMB_PA_MECVV= (AMB_PA_MECVV['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_AMB_PA_MECVV:.2f}%")

Percentatge de trastorns del son: 0.00%


In [688]:
ts_SENSE_PA= (SENSE_PA['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_SENSE_PA:.2f}%")

Percentatge de trastorns del son: 0.00%


In [689]:
ts_TOTAL= (TOTAL['Probl.son'] == 1.0).mean() * 100
print(f"Percentatge de trastorns del son: {ts_TOTAL:.2f}%")

Percentatge de trastorns del son: 0.00%


#### Pvalor

##### Normalitat

In [690]:
stat, p_value = shapiro(AMB_PA['Probl.son'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [691]:
stat, p_value = shapiro(AMB_PA_MECVV['Probl.son'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [692]:
stat, p_value = shapiro(SENSE_PA['Probl.son'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [693]:
stat, p_value = shapiro(TOTAL['Probl.son'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [694]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Probl.son'], columns=AMB_PA_MECVV['Probl.son']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.")

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i AMB_PA_MECVV.


In [695]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Probl.son'], columns=SENSE_PA['Probl.son']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i SENSE_PA.


In [696]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Probl.son'], columns=SENSE_PA['Probl.son']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [697]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA i TOTAL.


In [698]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en AMB_PA_MECVV i TOTAL.


In [699]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Probl.son'], columns=TOTAL['Probl.son']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de problemes de son en SENSE_PA i TOTAL.


## Dolor crònic

In [700]:
dc_AMB_PA= (AMB_PA['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [701]:
dc_AMB_PA_MECVV= (AMB_PA_MECVV['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_AMB_PA_MECVV:.2f}%")

Percentatge de dolor crònic: 0.00%


In [702]:
dc_SENSE_PA= (SENSE_PA['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_SENSE_PA:.2f}%")

Percentatge de dolor crònic: 0.00%


In [703]:
dc_TOTAL= (TOTAL['Dolor crònic'] == 1.0).mean() * 100
print(f"Percentatge de dolor crònic: {dc_TOTAL:.2f}%")

Percentatge de dolor crònic: 0.00%


#### Pvalor

##### Normalitat

In [704]:
stat, p_value = shapiro(AMB_PA['Dolor crònic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [705]:
stat, p_value = shapiro(AMB_PA_MECVV['Dolor crònic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [706]:
stat, p_value = shapiro(SENSE_PA['Dolor crònic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [707]:
stat, p_value = shapiro(TOTAL['Dolor crònic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [708]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=AMB_PA_MECVV['Dolor crònic']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i AMB_PA_MECVV.


In [709]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=SENSE_PA['Dolor crònic']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i SENSE_PA.


In [710]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Dolor crònic'], columns=SENSE_PA['Dolor crònic']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [711]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA i TOTAL.


In [712]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en AMB_PA_MECVV i TOTAL.


In [713]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Dolor crònic'], columns=TOTAL['Dolor crònic']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de dolor crònic en SENSE_PA i TOTAL.


## Iatrogènic

In [714]:
iatr_AMB_PA= (AMB_PA['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [715]:
iatr_AMB_PA_MECVV= (AMB_PA_MECVV['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_AMB_PA_MECVV:.2f}%")

Percentatge de iatrogènic: 0.00%


In [716]:
iatr_SENSE_PA= (SENSE_PA['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_SENSE_PA:.2f}%")

Percentatge de iatrogènic: 0.00%


In [717]:
iatr_TOTAL= (TOTAL['Iatrogènic'] == 1.0).mean() * 100
print(f"Percentatge de iatrogènic: {iatr_TOTAL:.2f}%")

Percentatge de iatrogènic: 0.00%


#### Pvalor

##### Normalitat

In [718]:
stat, p_value = shapiro(AMB_PA['Iatrogènic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [719]:
stat, p_value = shapiro(AMB_PA_MECVV['Iatrogènic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [720]:
stat, p_value = shapiro(SENSE_PA['Iatrogènic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [721]:
stat, p_value = shapiro(TOTAL['Iatrogènic'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [722]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=AMB_PA_MECVV['Iatrogènic']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i AMB_PA_MECVV.


In [723]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=SENSE_PA['Iatrogènic']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i SENSE_PA.


In [724]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Iatrogènic'], columns=SENSE_PA['Iatrogènic']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [725]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA i TOTAL.


In [726]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en AMB_PA_MECVV i TOTAL.


In [727]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Iatrogènic'], columns=TOTAL['Iatrogènic']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions d'iatrogènic en SENSE_PA i TOTAL.


## Restrenyiment

In [728]:
est_AMB_PA= (AMB_PA['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [729]:
est_AMB_PA_MECVV= (AMB_PA_MECVV['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_AMB_PA_MECVV:.2f}%")

Percentatge de restrenyiment: 0.00%


In [730]:
est_SENSE_PA= (SENSE_PA['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_SENSE_PA:.2f}%")

Percentatge de restrenyiment: 0.00%


In [731]:
est_TOTAL= (TOTAL['Restrenyiment'] == 1.0).mean() * 100
print(f"Percentatge de restrenyiment: {est_TOTAL:.2f}%")

Percentatge de restrenyiment: 0.00%


#### Pvalor

##### Normalitat

In [732]:
stat, p_value = shapiro(AMB_PA['Restrenyiment'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [733]:
stat, p_value = shapiro(AMB_PA_MECVV['Restrenyiment'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [734]:
stat, p_value = shapiro(SENSE_PA['Restrenyiment'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [735]:
stat, p_value = shapiro(TOTAL['Restrenyiment'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [736]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=AMB_PA_MECVV['Restrenyiment']) 
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12) 
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12) 
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12) 
 
if p_value_12 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i AMB_PA_MECVV.


In [737]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=SENSE_PA['Restrenyiment']) 
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13) 
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13) 
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13) 
 
if p_value_13 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i SENSE_PA.


In [738]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Restrenyiment'], columns=SENSE_PA['Restrenyiment']) 
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23) 
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23) 
 
if p_value_23 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i SENSE_PA.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [739]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14) 
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14) 
print("P-valor entre AMB_PA i TOTAL:", p_value_14) 
 
if p_value_14 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA i TOTAL.


In [740]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24) 
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24) 
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24) 
 
if p_value_24 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en AMB_PA_MECVV i TOTAL.


In [741]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Restrenyiment'], columns=TOTAL['Restrenyiment']) 
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34) 
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34) 
print("P-valor entre SENSE_PA i TOTAL:", p_value_34) 
 
if p_value_34 < 0.05: 
    print("Hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.") 
else: 
    print("No hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de restrenyiment en SENSE_PA i TOTAL.


# Historial mèdic/comorbilitats (aqui)

## Malalties CV

In [742]:
cv_AMB_PA= (AMB_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA:.2f}%")

Percentatge de problemes CV: 5.88%


In [743]:
cv_AMB_PA_MECVV= (AMB_PA_MECVV['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_AMB_PA_MECVV:.2f}%")

Percentatge de problemes CV: 0.00%


In [744]:
cv_SENSE_PA= (SENSE_PA['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_SENSE_PA:.2f}%")

Percentatge de problemes CV: 25.00%


In [745]:
cv_TOTAL= (TOTAL['CV'] == 1.0).mean() * 100
print(f"Percentatge de problemes CV: {cv_TOTAL:.2f}%")

Percentatge de problemes CV: 6.00%


#### Pvalor

##### Normalitat 

In [746]:
stat, p_value = shapiro(AMB_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2622
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [747]:
stat, p_value = shapiro(AMB_PA_MECVV['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [748]:
stat, p_value = shapiro(SENSE_PA['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [749]:
stat, p_value = shapiro(TOTAL['CV'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2548
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [750]:
contingency_table_12 = pd.crosstab(index=AMB_PA['CV'], columns=AMB_PA_MECVV['CV'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i AMB_PA_MECVV.


In [751]:
contingency_table_13 = pd.crosstab(index=AMB_PA['CV'], columns=SENSE_PA['CV'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.1875
P-valor entre AMB_PA i SENSE_PA: 0.6650055421020291
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i SENSE_PA.


In [752]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['CV'], columns=SENSE_PA['CV'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [753]:
contingency_table_14 = pd.crosstab(index=AMB_PA['CV'], columns=TOTAL['CV'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 3.7353515625
P-valor entre AMB_PA i TOTAL: 0.053272454982386926
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA i TOTAL.


In [754]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['CV'], columns=TOTAL['CV'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en AMB_PA_MECVV i TOTAL.


In [755]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['CV'], columns=TOTAL['CV'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties cardiovasculars en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 3.5555555555555554
P-valor entre SENSE_PA i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de malalties cardiovasculars en SENSE_PA i TOTAL.


## Malalties del cor

In [756]:
cora_AMB_PA= (AMB_PA['Probl.cor'] == 1.0).mean() * 100
print(f"Percentatge de malalties del cor: {cora_AMB_PA:.2f}%")

Percentatge de malalties del cor: 11.76%


In [757]:
cora_AMB_PA_MECVV= (AMB_PA_MECVV['Probl.cor'] == 1.0).mean() * 100
print(f"Percentatge de malalties del cor: {cora_AMB_PA_MECVV:.2f}%")

Percentatge de malalties del cor: 0.00%


In [758]:
cora_SENSE_PA= (SENSE_PA['Probl.cor'] == 1.0).mean() * 100
print(f"Percentatge de malalties del cor: {cora_SENSE_PA:.2f}%")

Percentatge de malalties del cor: 37.50%


In [759]:
cora_TOTAL= (TOTAL['Probl.cor'] == 1.0).mean() * 100
print(f"Percentatge de malalties del cor: {cora_TOTAL:.2f}%")

Percentatge de malalties del cor: 14.00%


#### Pvalor

##### Normalitat

In [760]:
stat, p_value = shapiro(AMB_PA['Probl.cor'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3853
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [761]:
stat, p_value = shapiro(AMB_PA_MECVV['Probl.cor'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [762]:
stat, p_value = shapiro(SENSE_PA['Probl.cor'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6412
Valor p = 0.0005
El nombre de files no segueix una distribució normal.


In [763]:
stat, p_value = shapiro(TOTAL['Probl.cor'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4124
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [764]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Probl.cor'], columns=AMB_PA_MECVV['Probl.cor'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i AMB_PA_MECVV.


In [765]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Probl.cor'], columns=SENSE_PA['Probl.cor'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i SENSE_PA.


In [766]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Probl.cor'], columns=SENSE_PA['Probl.cor'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [767]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Probl.cor'], columns=TOTAL['Probl.cor'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 8.731388888888889
P-valor entre AMB_PA i TOTAL: 0.0031277815159609333
Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA i TOTAL.


In [768]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Probl.cor'], columns=TOTAL['Probl.cor'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malalties del cor en AMB_PA_MECVV i TOTAL.


In [769]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Probl.cor'], columns=TOTAL['Probl.cor'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties del cor en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties del cor en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.302222222222222
P-valor entre SENSE_PA i TOTAL: 0.0380626070324449
Hi ha diferències significatives entre les distribucions de malalties del cor en SENSE_PA i TOTAL.


## Malalties neurodegeneratives

In [770]:
nd_AMB_PA= (AMB_PA['Neurodegeneratives'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 0.00%


In [771]:
nd_AMB_PA_MECVV= (AMB_PA_MECVV['Neurodegeneratives'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_AMB_PA_MECVV:.2f}%")

Percentatge de malalties neurodegeneratives: 12.50%


In [772]:
nd_SENSE_PA= (SENSE_PA['Neurodegeneratives'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_SENSE_PA:.2f}%")

Percentatge de malalties neurodegeneratives: 12.50%


In [773]:
nd_TOTAL= (TOTAL['Neurodegeneratives'] == 1.0).mean() * 100
print(f"Percentatge de malalties neurodegeneratives: {nd_TOTAL:.2f}%")

Percentatge de malalties neurodegeneratives: 4.00%


#### Pvalor

##### Normalitat

In [774]:
stat, p_value = shapiro(AMB_PA['Neurodegeneratives'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [775]:
stat, p_value = shapiro(AMB_PA_MECVV['Neurodegeneratives'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [776]:
stat, p_value = shapiro(SENSE_PA['Neurodegeneratives'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [777]:
stat, p_value = shapiro(TOTAL['Neurodegeneratives'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1982
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [778]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Neurodegeneratives'], columns=AMB_PA_MECVV['Neurodegeneratives'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i AMB_PA_MECVV.


In [779]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Neurodegeneratives'], columns=SENSE_PA['Neurodegeneratives'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i SENSE_PA.


In [780]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Neurodegeneratives'], columns=SENSE_PA['Neurodegeneratives'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [781]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Neurodegeneratives'], columns=TOTAL['Neurodegeneratives'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA i TOTAL.


In [782]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Neurodegeneratives'], columns=TOTAL['Neurodegeneratives'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en AMB_PA_MECVV i TOTAL.


In [783]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Neurodegeneratives'], columns=TOTAL['Neurodegeneratives'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de malalties neurodegeneratives en SENSE_PA i TOTAL.


## Diabetis mellitus

In [784]:
dm_AMB_PA= (AMB_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetis mellitus: {dm_AMB_PA:.2f}%")

Percentatge de diabetis mellitus: 23.53%


In [785]:
dm_AMB_PA_MECVV= (AMB_PA_MECVV['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetis mellitus: {dm_AMB_PA_MECVV:.2f}%")

Percentatge de diabetis mellitus: 25.00%


In [786]:
dm_SENSE_PA= (SENSE_PA['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetis mellitus: {dm_SENSE_PA:.2f}%")

Percentatge de diabetis mellitus: 25.00%


In [787]:
dm_TOTAL= (TOTAL['DM'] == 1.0).mean() * 100
print(f"Percentatge de diabetis mellitus: {dm_TOTAL:.2f}%")

Percentatge de diabetis mellitus: 26.00%


#### Pvalor 

##### Normalitat

In [788]:
stat, p_value = shapiro(AMB_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [789]:
stat, p_value = shapiro(AMB_PA_MECVV['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [790]:
stat, p_value = shapiro(SENSE_PA['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5659
Valor p = 0.0001
El nombre de files no segueix una distribució normal.


In [791]:
stat, p_value = shapiro(TOTAL['DM'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5472
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [792]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Neurodegeneratives'], columns=AMB_PA_MECVV['Neurodegeneratives'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i AMB_PA_MECVV.


In [793]:
contingency_table_13 = pd.crosstab(index=AMB_PA['DM'], columns=SENSE_PA['DM'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i SENSE_PA.


In [794]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['DM'], columns=SENSE_PA['DM'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [795]:
contingency_table_14 = pd.crosstab(index=AMB_PA['DM'], columns=TOTAL['DM'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA i TOTAL.


In [796]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['DM'], columns=TOTAL['DM'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 3.5555555555555554
P-valor entre AMB_PA_MECVV i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de diabetis mellitus en AMB_PA_MECVV i TOTAL.


In [797]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['DM'], columns=TOTAL['DM'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de diabetis mellitus en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de diabetis mellitus en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 3.5555555555555554
P-valor entre SENSE_PA i TOTAL: 0.05934643879191998
No hi ha diferències significatives entre les distribucions de diabetis mellitus en SENSE_PA i TOTAL.


## Hepatopaties

In [798]:
hp_AMB_PA= (AMB_PA['Hepatopaties'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA:.2f}%")

Percentatge de hepatopaties: 11.76%


In [799]:
hp_AMB_PA_MECVV= (AMB_PA_MECVV['Hepatopaties'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_AMB_PA_MECVV:.2f}%")

Percentatge de hepatopaties: 12.50%


In [800]:
hp_SENSE_PA= (SENSE_PA['Hepatopaties'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_SENSE_PA:.2f}%")

Percentatge de hepatopaties: 0.00%


In [801]:
hp_TOTAL= (TOTAL['Hepatopaties'] == 1.0).mean() * 100
print(f"Percentatge de hepatopaties: {hp_TOTAL:.2f}%")

Percentatge de hepatopaties: 6.00%


#### Pvalor

##### Normalitat

In [802]:
stat, p_value = shapiro(AMB_PA['Hepatopaties'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.3853
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [803]:
stat, p_value = shapiro(AMB_PA_MECVV['Hepatopaties'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [804]:
stat, p_value = shapiro(SENSE_PA['Hepatopaties'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [805]:
stat, p_value = shapiro(TOTAL['Hepatopaties'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2548
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [806]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Hepatopaties'], columns=AMB_PA_MECVV['Hepatopaties'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.9600000000000002
P-valor entre AMB_PA i AMB_PA_MECVV: 0.32718687779030275
No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i AMB_PA_MECVV.


In [807]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Hepatopaties'], columns=SENSE_PA['Hepatopaties'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i SENSE_PA.


In [808]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Hepatopaties'], columns=SENSE_PA['Hepatopaties'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [809]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Hepatopaties'], columns=TOTAL['Hepatopaties'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 8.731388888888889
P-valor entre AMB_PA i TOTAL: 0.0031277815159609333
Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA i TOTAL.


In [810]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Hepatopaties'], columns=TOTAL['Hepatopaties'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 1.469387755102041
P-valor entre AMB_PA_MECVV i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de hepatopaties en AMB_PA_MECVV i TOTAL.


In [811]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Hepatopaties'], columns=TOTAL['Hepatopaties'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de hepatopaties en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de hepatopaties en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de hepatopaties en SENSE_PA i TOTAL.


## Neoplàsies

In [812]:
neopl_AMB_PA= (AMB_PA['Neoplàsies'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsies: {neopl_AMB_PA:.2f}%")

Percentatge de neoplàsies: 0.00%


In [813]:
neopl_AMB_PA_MECVV= (AMB_PA_MECVV['Neoplàsies'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsies: {neopl_AMB_PA_MECVV:.2f}%")

Percentatge de neoplàsies: 0.00%


In [814]:
neopl_SENSE_PA= (SENSE_PA['Neoplàsies'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsies: {neopl_SENSE_PA:.2f}%")

Percentatge de neoplàsies: 12.50%


In [815]:
neopl_TOTAL= (TOTAL['Neoplàsies'] == 1.0).mean() * 100
print(f"Percentatge de neoplàsies: {neopl_TOTAL:.2f}%")

Percentatge de neoplàsies: 6.00%


#### Pvalor

##### Normalitat

In [816]:
stat, p_value = shapiro(AMB_PA['Neoplàsies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [817]:
stat, p_value = shapiro(AMB_PA_MECVV['Neoplàsies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [818]:
stat, p_value = shapiro(SENSE_PA['Neoplàsies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.4184
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [819]:
stat, p_value = shapiro(TOTAL['Neoplàsies'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.2548
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [820]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Neoplàsies'], columns=AMB_PA_MECVV['Neoplàsies'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i AMB_PA_MECVV.


In [821]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Neoplàsies'], columns=SENSE_PA['Neoplàsies'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i SENSE_PA.


In [822]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Neoplàsies'], columns=SENSE_PA['Neoplàsies'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [823]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Neoplàsies'], columns=TOTAL['Neoplàsies'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA i TOTAL.


In [824]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Neoplàsies'], columns=TOTAL['Neoplàsies'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de neoplàsies en AMB_PA_MECVV i TOTAL.


In [825]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Neoplàsies'], columns=TOTAL['Neoplàsies'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de neoplàsies en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de neoplàsies en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 1.469387755102041
P-valor entre SENSE_PA i TOTAL: 0.22544231699450956
No hi ha diferències significatives entre les distribucions de neoplàsies en SENSE_PA i TOTAL.


## Malaltia crònica renal (creatinina > 1.5mg/dL)

Donat que aquesta columna és de tipus objecte, primer la passaré a tipus numèric per poder operar amb ella

In [826]:
AMB_PA.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(AMB_PA['Creatinina'], errors='coerce')
AMB_PA_MECVV.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(AMB_PA_MECVV['Creatinina'], errors='coerce')
SENSE_PA.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(SENSE_PA['Creatinina'], errors='coerce')
TOTAL.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(TOTAL['Creatinina'], errors='coerce')

C:\Users\User\AppData\Local\Temp\ipykernel_5224\3778722507.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AMB_PA.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(AMB_PA['Creatinina'], errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_5224\3778722507.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  AMB_PA_MECVV.loc[:, 'Creatinina >1.5 binària'] = pd.to_numeric(AMB_PA_MECVV['Creatinina'], errors='coerce')
C:\Users\User\AppData\Local\Temp\ipykernel_5224\3778722507.py:3: SettingWithCopyWarnin

In [827]:
ecr_AMB_PA= (AMB_PA['Creatinina >1.5 binària'] == 1.0).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [882]:
ecr_AMB_PA_MECVV= (AMB_PA_MECVV['Creatinina >1.5 binària'] == 1.0).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_AMB_PA_MECVV:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [883]:
ecr_SENSE_PA= (SENSE_PA['Creatinina >1.5 binària'] == 1.0).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_SENSE_PA:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 0.00%


In [884]:
ecr_TOTAL= (TOTAL['Creatinina >1.5 binària'] == 1.0).mean() * 100
print(f"Percentatge de malaltia crònica renal amb creatinina >1.5: {ecr_TOTAL:.2f}%")

Percentatge de malaltia crònica renal amb creatinina >1.5: 2.00%


#### Pvalor

##### Normalitat

In [885]:
stat, p_value = shapiro(AMB_PA['Creatinina >1.5 binària'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9549
Valor p = 0.7439
El nombre de files segueix una distribució normal


In [886]:
stat, p_value = shapiro(AMB_PA_MECVV['Creatinina >1.5 binària'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9030
Valor p = 0.3952
El nombre de files segueix una distribució normal


In [887]:
stat, p_value = shapiro(SENSE_PA['Creatinina >1.5 binària'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.9674
Valor p = 0.8746
El nombre de files segueix una distribució normal


In [888]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Creatinina >1.5 binària'], columns=AMB_PA_MECVV['Creatinina >1.5 binària'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 6.000000000000001
P-valor entre AMB_PA i AMB_PA_MECVV: 0.19914827347145564
No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i AMB_PA_MECVV.


In [889]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Creatinina >1.5 binària'], columns=SENSE_PA['Creatinina >1.5 binària'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de  malaltia crònica renal amb creatinina >1.5 en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de  malaltia crònica renal amb creatinina >1.5 en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 6.000000000000001
P-valor entre AMB_PA i SENSE_PA: 0.19914827347145564
No hi ha diferències significatives entre les distribucions de  malaltia crònica renal amb creatinina >1.5 en AMB_PA i SENSE_PA.


In [890]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Creatinina >1.5 binària'], columns=SENSE_PA['Creatinina >1.5 binària'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [891]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Creatinina >1.5 binària'], columns=TOTAL['Creatinina >1.5 binària'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 71.99999999999999
P-valor entre AMB_PA i TOTAL: 0.23025670024966835
No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA i TOTAL.


In [892]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Creatinina >1.5 binària'], columns=TOTAL['Creatinina >1.5 binària'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 6.000000000000001
P-valor entre AMB_PA_MECVV i TOTAL: 0.19914827347145564
No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en AMB_PA_MECVV i TOTAL.


In [893]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Creatinina >1.5 binària'], columns=TOTAL['Creatinina >1.5 binària'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 30.00000000000001
P-valor entre SENSE_PA i TOTAL: 0.22428900483440337
No hi ha diferències significatives entre les distribucions de malaltia crònica renal amb creatinina >1.5 en SENSE_PA i TOTAL.


## Fallida renal aguda (o insuficiència renal aguda)

In [836]:
ira_AMB_PA= (AMB_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA:.2f}%")

Percentatge de fallida renal aguda: 23.53%


In [837]:
ira_AMB_PA_MECVV= (AMB_PA_MECVV['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_AMB_PA_MECVV:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [838]:
ira_SENSE_PA= (SENSE_PA['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_SENSE_PA:.2f}%")

Percentatge de fallida renal aguda: 50.00%


In [894]:
ira_TOTAL= (TOTAL['ARF'] == 1.0).mean() * 100
print(f"Percentatge de fallida renal aguda: {ira_TOTAL:.2f}%")

Percentatge de fallida renal aguda: 26.00%


#### Pvalor

##### Normalitat

In [839]:
stat, p_value = shapiro(AMB_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5327
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [840]:
stat, p_value = shapiro(AMB_PA_MECVV['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [841]:
stat, p_value = shapiro(SENSE_PA['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.6647
Valor p = 0.0009
El nombre de files no segueix una distribució normal.


In [895]:
stat, p_value = shapiro(TOTAL['ARF'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.5472
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [896]:
contingency_table_12 = pd.crosstab(index=AMB_PA['ARF'], columns=AMB_PA_MECVV['ARF'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 2.6666666666666665
P-valor entre AMB_PA i AMB_PA_MECVV: 0.10247043485974942
No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i AMB_PA_MECVV.


In [897]:
contingency_table_13 = pd.crosstab(index=AMB_PA['ARF'], columns=SENSE_PA['ARF'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i SENSE_PA.


In [898]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['ARF'], columns=SENSE_PA['ARF'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [899]:
contingency_table_14 = pd.crosstab(index=AMB_PA['ARF'], columns=TOTAL['ARF'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 11.89654215976331
P-valor entre AMB_PA i TOTAL: 0.0005623824703676534
Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA i TOTAL.


In [900]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['ARF'], columns=TOTAL['ARF'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 4.5
P-valor entre AMB_PA_MECVV i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de fallida renal aguda en AMB_PA_MECVV i TOTAL.


In [901]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['ARF'], columns=TOTAL['ARF'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de fallida renal aguda en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de fallida renal aguda en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 4.5
P-valor entre SENSE_PA i TOTAL: 0.033894853524689295
Hi ha diferències significatives entre les distribucions de fallida renal aguda en SENSE_PA i TOTAL.


## Marejos/síncope/lipotímia/reacció vasovagal

In [903]:
dizsyn_AMB_PA= (AMB_PA['Marejos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [904]:
dizsyn_AMB_PA_MECVV= (AMB_PA_MECVV['Marejos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_AMB_PA_MECVV:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [905]:
dizsyn_SENSE_PA= (SENSE_PA['Marejos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_SENSE_PA:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 0.00%


In [906]:
dizsyn_TOTAL= (TOTAL['Marejos'] == 1.0).mean() * 100
print(f"Percentatge de marejos/síncope/lipotímia/reacció vasovagal: {dizsyn_TOTAL:.2f}%")

Percentatge de marejos/síncope/lipotímia/reacció vasovagal: 2.00%


#### Pvalor

##### Normalitat

In [907]:
stat, p_value = shapiro(AMB_PA['Marejos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


C:\Users\User\PycharmProjects\PRACTICUM\venv\Lib\site-packages\scipy\stats\_axis_nan_policy.py:531: UserWarning: scipy.stats.shapiro: Input data has range zero. The results may not be accurate.
  res = hypotest_fun_out(*samples, **kwds)


In [908]:
stat, p_value = shapiro(AMB_PA_MECVV['Marejos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [909]:
stat, p_value = shapiro(SENSE_PA['Marejos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [910]:
stat, p_value = shapiro(TOTAL['Marejos'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal.")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 0.1254
Valor p = 0.0000
El nombre de files no segueix una distribució normal.


In [911]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Marejos'], columns=AMB_PA_MECVV['Marejos'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i AMB_PA_MECVV.


In [912]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Marejos'], columns=SENSE_PA['Marejos'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA i SENSE_PA.


In [923]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Marejos'], columns=SENSE_PA['Marejos'])
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció vasovagal en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [914]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Marejos'], columns=TOTAL['Marejos'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA i TOTAL.


In [915]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Marejos'], columns=TOTAL['Marejos'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en AMB_PA_MECVV i TOTAL.


In [916]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Marejos'], columns=TOTAL['Marejos'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de marejos/síncope/lipotímia/reacció en SENSE_PA i TOTAL.


## VIH

In [855]:
vih_AMB_PA= (AMB_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA:.2f}%")

Percentatge de VIH: 0.00%


In [856]:
vih_AMB_PA_MECVV= (AMB_PA_MECVV['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_AMB_PA_MECVV:.2f}%")

Percentatge de VIH: 0.00%


In [857]:
vih_SENSE_PA= (SENSE_PA['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_SENSE_PA:.2f}%")

Percentatge de VIH: 0.00%


In [917]:
vih_TOTAL= (TOTAL['VIH'] == 1.0).mean() * 100
print(f"Percentatge de VIH: {vih_TOTAL:.2f}%")

Percentatge de VIH: 0.00%


#### Pvalor

##### Normalitat

In [920]:
stat, p_value = shapiro(AMB_PA['VIH'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [859]:
stat, p_value = shapiro(AMB_PA_MECVV['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [860]:
stat, p_value = shapiro(SENSE_PA['VIH'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [919]:
stat, p_value = shapiro(TOTAL['VIH'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [924]:
contingency_table_12 = pd.crosstab(index=AMB_PA['VIH'], columns=AMB_PA_MECVV['VIH'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i AMB_PA_MECVV.


In [925]:
contingency_table_13 = pd.crosstab(index=AMB_PA['VIH'], columns=SENSE_PA['VIH'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i SENSE_PA.")

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i SENSE_PA.


In [926]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['VIH'], columns=SENSE_PA['VIH'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [927]:
contingency_table_14 = pd.crosstab(index=AMB_PA['VIH'], columns=TOTAL['VIH'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de VIH en AMB_PA i TOTAL.


In [928]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['VIH'], columns=TOTAL['VIH'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de VIH en AMB_PA_MECVV i TOTAL.


In [929]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['VIH'], columns=TOTAL['VIH'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de VIH en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de VIH en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de VIH en SENSE_PA i TOTAL.


## Psicosi

In [930]:
psi_AMB_PA= (AMB_PA['Psicosi'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {vih_AMB_PA:.2f}%")

Percentatge de psicosi: 0.00%


In [931]:
psi_AMB_PA_MECVV= (AMB_PA_MECVV['Psicosi'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_AMB_PA_MECVV:.2f}%")

Percentatge de psicosi: 0.00%


In [932]:
psi_SENSE_PA= (SENSE_PA['Psicosi'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_SENSE_PA:.2f}%")

Percentatge de psicosi: 0.00%


In [933]:
psi_TOTAL= (TOTAL['Psicosi'] == 1.0).mean() * 100
print(f"Percentatge de psicosi: {psi_TOTAL:.2f}%")

Percentatge de psicosi: 0.00%


#### Pvalor 

##### Normalitat

In [936]:
stat, p_value = shapiro(AMB_PA['Psicosi'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [935]:
stat, p_value = shapiro(AMB_PA_MECVV['Psicosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [937]:
stat, p_value = shapiro(SENSE_PA['Psicosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [938]:
stat, p_value = shapiro(TOTAL['Psicosi'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [939]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Psicosi'], columns=AMB_PA_MECVV['Psicosi'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i AMB_PA_MECVV.


In [940]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Psicosi'], columns=SENSE_PA['Psicosi'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i SENSE_PA.


In [941]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Psicosi'], columns=SENSE_PA['Psicosi'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [942]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Psicosi'], columns=TOTAL['Psicosi'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA i TOTAL.


In [943]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Psicosi'], columns=TOTAL['Psicosi'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en AMB_PA_MECVV i TOTAL.


In [944]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Psicosi'], columns=TOTAL['Psicosi'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de psicosi en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de psicosi en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de psicosi en SENSE_PA i TOTAL.


## Deficiències nutricionals

In [945]:
nutri_AMB_PA= (AMB_PA['Def.nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [946]:
nutri_AMB_PA_MECVV= (AMB_PA_MECVV['Def.nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_AMB_PA_MECVV:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [947]:
nutri_SENSE_PA= (SENSE_PA['Def.nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_SENSE_PA:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


In [948]:
nutri_TOTAL= (TOTAL['Def.nutri'] == 1.0).mean() * 100
print(f"Percentatge de deficiències nutricionals: {nutri_TOTAL:.2f}%")

Percentatge de deficiències nutricionals: 0.00%


#### Pvalor

##### Normalitat

In [950]:
stat, p_value = shapiro(AMB_PA['Def.nutri'].dropna())
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [951]:
stat, p_value = shapiro(AMB_PA_MECVV['Def.nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [952]:
stat, p_value = shapiro(SENSE_PA['Def.nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [953]:
stat, p_value = shapiro(TOTAL['Def.nutri'])
print(f"Estadístic W = {stat:.4f}")
print(f"Valor p = {p_value:.4f}")

alpha = 0.05
if p_value < alpha:
    print("El nombre de files no segueix una distribució normal")
else:
    print("El nombre de files segueix una distribució normal")

Estadístic W = 1.0000
Valor p = 1.0000
El nombre de files segueix una distribució normal


In [954]:
contingency_table_12 = pd.crosstab(index=AMB_PA['Def.nutri'], columns=AMB_PA_MECVV['Def.nutri'])  
 
chi2_12, p_value_12, dof_12, expected_12 = chi2_contingency(contingency_table_12)  
 
print("Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV:", chi2_12)  
print("P-valor entre AMB_PA i AMB_PA_MECVV:", p_value_12)  
 
if p_value_12 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i AMB_PA_MECVV.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i AMB_PA_MECVV.") 

Estadístic xi-quadrat entre AMB_PA i AMB_PA_MECVV: 0.0
P-valor entre AMB_PA i AMB_PA_MECVV: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i AMB_PA_MECVV.


In [955]:
contingency_table_13 = pd.crosstab(index=AMB_PA['Def.nutri'], columns=SENSE_PA['Def.nutri'])  
 
chi2_13, p_value_13, dof_13, expected_13 = chi2_contingency(contingency_table_13)  
 
print("Estadístic xi-quadrat entre AMB_PA i SENSE_PA:", chi2_13)  
print("P-valor entre AMB_PA i SENSE_PA:", p_value_13)  
 
if p_value_13 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i SENSE_PA.") 

Estadístic xi-quadrat entre AMB_PA i SENSE_PA: 0.0
P-valor entre AMB_PA i SENSE_PA: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i SENSE_PA.


In [956]:
contingency_table_23 = pd.crosstab(index=AMB_PA_MECVV['Def.nutri'], columns=SENSE_PA['Def.nutri'])  
 
chi2_23, p_value_23, dof_23, expected_23 = chi2_contingency(contingency_table_23)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECCV i SENSE_PA:", chi2_23)  
print("P-valor entre AMB_PA_MECVV i SENSE_PA:", p_value_23)  
 
if p_value_23 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i SENSE_PA.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i SENSE_PA.") 

ValueError: No data; `observed` has size 0.

In [957]:
contingency_table_14 = pd.crosstab(index=AMB_PA['Def.nutri'], columns=TOTAL['Def.nutri'])  
 
chi2_14, p_value_14, dof_14, expected_14 = chi2_contingency(contingency_table_14)  
 
print("Estadístic xi-quadrat entre AMB_PA i TOTAL:", chi2_14)  
print("P-valor entre AMB_PA i TOTAL:", p_value_14)  
 
if p_value_14 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA i TOTAL: 0.0
P-valor entre AMB_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA i TOTAL.


In [958]:
contingency_table_24 = pd.crosstab(index=AMB_PA_MECVV['Def.nutri'], columns=TOTAL['Def.nutri'])  
 
chi2_24, p_value_24, dof_24, expected_24 = chi2_contingency(contingency_table_24)  
 
print("Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL:", chi2_24)  
print("P-valor entre AMB_PA_MECVV i TOTAL:", p_value_24)  
 
if p_value_24 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i TOTAL.") 

Estadístic xi-quadrat entre AMB_PA_MECVV i TOTAL: 0.0
P-valor entre AMB_PA_MECVV i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en AMB_PA_MECVV i TOTAL.


In [959]:
contingency_table_34 = pd.crosstab(index=SENSE_PA['Def.nutri'], columns=TOTAL['Def.nutri'])  
 
chi2_34, p_value_34, dof_34, expected_34 = chi2_contingency(contingency_table_34)  
 
print("Estadístic xi-quadrat entre SENSE_PA i TOTAL:", chi2_34)  
print("P-valor entre SENSE_PA i TOTAL:", p_value_34)  
 
if p_value_34 < 0.05:  
   print("Hi ha diferències significatives entre les distribucions de deficiències nutricionals en SENSE_PA i TOTAL.")  
else:  
   print("No hi ha diferències significatives entre les distribucions de deficiències nutricionals en SENSE_PA i TOTAL.") 

Estadístic xi-quadrat entre SENSE_PA i TOTAL: 0.0
P-valor entre SENSE_PA i TOTAL: 1.0
No hi ha diferències significatives entre les distribucions de deficiències nutricionals en SENSE_PA i TOTAL.
